# Convolutional Neural Network

### Importing the libraries

In [13]:
import tensorflow as  tf
from keras.preprocessing.image import ImageDataGenerator

In [14]:
tf.__version__

'2.7.0'

## Part 1 - Data Preprocessing
- Applying some transformation on training set to avoid overfitting.
- Image Augmentation
- After transformation we get variety of images in training set.
- Transformation we'll be using : 
  - `shear_range`
  - `zoom_range`
  - `horizontal_flip`
- Normalization is compulsory in CNN.

### Preprocessing the Training set
- Image augmentation is only applied on training set.

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # feature scalling | Normalization
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',     # path
    target_size=(64,64),
    batch_size=32,
    class_mode='binary',
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set
- Like, what we have done in previous section with `fit_transform` (training) and `transform` (testing). we are doing just the same thing here.

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary',
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [17]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution
-  we have to specify inputshape in our first layer
  - if we are working with black and white images then this input shape should be `(64,64,1)`. but here we have RGB values so we are considering it to be `3`.

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling
-  applying `MAX` pooling

In [19]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a second convolutional layer

In [20]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))    # `input_shape` parameter is removed as it is only required in first layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Step 3 - Flattening

In [21]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection
- `units` : number of hidden neurons
- `activation` : as long as we haven't reached the output layer, we should keep using `relu` activation *(Rectified Linear Unit)*

In [22]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer
- `units` : Number of hidden units are **1** here, as we are doing binary classification of **cats/dogs**.
- for output layer we should use `Sigmoid` as we are  doing binary classification. *(for multiclass classification, we should use `softmax` activation function)*

In [23]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [24]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set
- 250 means, we have 8000 images from which we made 32 images' batch so `8000/32=250`

In [26]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 45s 178ms/step - loss: 0.6700 - accuracy: 0.5924 - val_loss: 0.6377 - val_accuracy: 0.6235
Epoch 2/25
250/250 [==============================] - 36s 144ms/step - loss: 0.6103 - accuracy: 0.6671 - val_loss: 0.5598 - val_accuracy: 0.7150
Epoch 3/25
250/250 [==============================] - 41s 164ms/step - loss: 0.5634 - accuracy: 0.7094 - val_loss: 0.5276 - val_accuracy: 0.7415
Epoch 4/25
250/250 [==============================] - 40s 160ms/step - loss: 0.5272 - accuracy: 0.7352 - val_loss: 0.5253 - val_accuracy: 0.7400
Epoch 5/25
250/250 [==============================] - 37s 147ms/step - loss: 0.5083 - accuracy: 0.7508 - val_loss: 0.5420 - val_accuracy: 0.7210
Epoch 6/25
250/250 [==============================] - 37s 147ms/step - loss: 0.4865 - accuracy: 0.7646 - val_loss: 0.5201 - val_accuracy: 0.7430
Epoch 7/25
250/250 [==============================] - 35s 141ms/step - loss: 0.4692 - accuracy: 0.7747 - val_loss: 0.5198 - val_ac

## Part 4 - Making a single prediction
- as we have changed the `target_size` of the image while training, we also have to change the size of the to-be-predicted image.
- as we have given images in batches of size `32` in training, we also have to give this one image in batch format in order to predict the class.

- here : 
  - `1` : Dog
  - `0` : Cat

In [29]:
import numpy as np
from keras.preprocessing import image 
def predict_image(img_path):
  test_image = image.load_img(img_path, target_size=(64, 64))
  test_image = image.img_to_array(test_image)   # this is a numpy array
  test_image = np.expand_dims(test_image, axis=0)   # adding batch dimention as first dimention
  result = cnn.predict(test_image)
  training_set.class_indices

  # here this `result` variable also has batch dimention.
  if result[0][0]==1:
    prediction='dog'
  else:
    prediction='cat'
  return prediction

In [30]:
print('1',predict_image('dataset/single_prediction/cat_or_dog_1.jpg'))
print('2',predict_image('dataset/single_prediction/cat_or_dog_2.jpg'))

1 dog
2 cat
